# Meera Retailer Campaign - Random Pick

## Objective
* Choose random retailers satisfying certain criteria providing weights based on the purchasing ability of retailers
* Choose the number of random retailers based on the weightage of number of retailers under each AM

## Steps

1. Import packages
2. Import data
3. Remove null
4. Occurence of Retailer
5. Weightage for AM
6. Split data based on AM's
7. Function to generate random Rtr's with AM weightage

### Import packages

In [59]:
from ckpackages import azsql 
import numpy as np
import pandas as pd
import random

### Import data

In [60]:
Query = 'select * from v_PC_MeeraRetailerCampaign' 
#RSC Transactions for Meera Powder
data = azsql.callstatement(Query)

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   RtrCode         766 non-null    object        
 1   RtrName         766 non-null    object        
 2   RtrGroupCde     761 non-null    object        
 3   DistName        766 non-null    object        
 4   BillNo          766 non-null    object        
 5   InvDate         766 non-null    datetime64[ns]
 6   Brand           766 non-null    object        
 7   PrdName         766 non-null    object        
 8   StateName       0 non-null      object        
 9   ZM              766 non-null    object        
 10  AMName          766 non-null    object        
 11  TSOName         766 non-null    object        
 12  TotalQty        766 non-null    float64       
 13  PolypouchCount  766 non-null    float64       
 14  FinYr           766 non-null    object        
 15  JC    

In [62]:
data.head()

,RtrCode,RtrName,RtrGroupCde,DistName,BillNo,InvDate,Brand,PrdName,StateName,ZM,AMName,TSOName,TotalQty,PolypouchCount,FinYr,JC,JCWk
0,TNLUM001-SLM0011553239845,JR MALIGAI,PC-KIRANA/GENERAL STORES,STAR MARKETING,CKBIL2005953,2020-09-30,Meera Powder & Paste,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,None,C PERUMAL,SYED IBRAHIM,R JAIGANESH,150.0,1.000000,2020-21,JC07,WK02
1,GT0100020067820002006782-SLM0011562060211,M.AHAMED&CO,PC-KIRANA/GENERAL STORES,SRI CHANDRA STORES,CKBIL2000222,2020-09-30,Meera Powder & Paste,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,None,C PERUMAL,SYED IBRAHIM,ELANGO VELUSAMY,2700.0,18.000000,2020-21,JC07,WK02
2,S100000941550670081,Sundha general store,PC-KIRANA/GENERAL STORES,JAI AGENCIES,CKBIL2001933,2020-09-30,Meera Powder & Paste,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,None,C PERUMAL,BALAJI S,IMMANUEL RAJA J,1350.0,9.000000,2020-21,JC07,WK02
3,GT0100020001540002000154-SLM0011556707922,sri suntha agency,PC-WHOLESALE,ESWARI AGENCIES,CKBIL2001633,2020-10-04,Meera Powder & Paste,MEERA ADVANCE HW POWDER 5.6 GM 1350 PCS,None,C PERUMAL,SYED IBRAHIM,R JAIGANESH,150.0,1.000000,2020-21,JC07,WK03
4,GT0100020000770002000077-SLM0011560947175,mansarovar,PC-WHOLESALE,ARIHANT DISTRIBUTORS,CKBIL2000405,2020-09-30,Meera Powder & Paste,MEERA ADVANCE HW POWDER 40GM REFILL 100P,None,C PERUMAL,BALAJI S,R RAJA,500.0,3.333333,2020-21,JC07,WK02


In [63]:
# Data only for Latest date
data = data[data['InvDate'] == data['InvDate'].max()]

### Remove null

In [64]:
# Retailers without AM
print(data['AMName'].isnull().sum())
data[data['AMName'].isnull()]

0


,RtrCode,RtrName,RtrGroupCde,DistName,BillNo,InvDate,Brand,PrdName,StateName,ZM,AMName,TSOName,TotalQty,PolypouchCount,FinYr,JC,JCWk


In [65]:
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

### Occurence of Retailer

In [66]:
# List of retailer
rt = data['RtrCode'].tolist()
rt.sort()
rtr = pd.Series(rt).value_counts()
rtr_list = pd.DataFrame(rtr,columns = ['retailer'] )
rtr_list['retailer']

TNJTR006-SLM0021552892243                     1
S100000701550583816                           1
S100000601551088512                           1
S100000351550925744                           1
TNNAA003-SLM0011553499862                     1
                                             ..
GT0100020035390002003539-SLM00231574664081    1
S100000161563351070                           1
TNNAA003-SLM0011553515191                     1
S100000021550647196                           1
S100000211564988007                           1
Name: retailer, Length: 143, dtype: int64

In [67]:
# Maximum occurence of retailer
rtr_list['retailer'].max()

1

In [68]:
rtr_master = data[['RtrCode','RtrName','DistName','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()

In [69]:
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

In [70]:
# Choose number of retailers to choose
pick = 45

### Weightage for AM

In [71]:
# Percentage of Retailers across AM-Wise
TotalRtrCount = data['RtrCode'].count()
AM_RTR_WGT = data.groupby('AMName')['RtrCode'].count().reset_index()
AM_RTR_WGT['WGT%'] = np.ceil((AM_RTR_WGT['RtrCode']/TotalRtrCount) * pick)

In [72]:
TotalRtrCount

143

In [74]:
AM_RTR_WGT

,AMName,RtrCode,WGT%
0,BALAJI S,83,27.0
1,SAFIULLAH,29,10.0
2,SYED IBRAHIM,27,9.0
3,V BHARATHI,4,2.0


### Split dataframe on AM's

In [75]:
# Select required rows
data1 = data[['ZM','AMName','RtrCode','RtrName','TotalQty','PolypouchCount']]

In [76]:
data2 = [rows for _, rows in data1.groupby('AMName')]
#data2 = [{gender: rows for gender, rows in data1.groupby('AMName')}]

In [77]:
data2[0]

,ZM,AMName,RtrCode,RtrName,TotalQty,PolypouchCount
109,C PERUMAL,BALAJI S,S100000361552893510,Ganesh store,150.0,1.0
137,C PERUMAL,BALAJI S,S100000361552897824,D. K. P stores,150.0,1.0
164,C PERUMAL,BALAJI S,S100000361552900061,Mariyappan Wholesale,150.0,1.0
189,C PERUMAL,BALAJI S,S100000361552900907,Lakshmi MALLIGAI,150.0,1.0
198,C PERUMAL,BALAJI S,S100000481551172555,SARAVANA STORE,150.0,1.0
...,...,...,...,...,...,...
750,C PERUMAL,BALAJI S,S100000431551682220,MADHAN STORE,150.0,1.0
752,C PERUMAL,BALAJI S,S100000431551687714,ANNAI STORE,450.0,3.0
754,C PERUMAL,BALAJI S,S100000461551077596,SRI ARUNA STORE,150.0,1.0
756,C PERUMAL,BALAJI S,S100000461551085711,SRI SAIRAM STORE,150.0,1.0


In [79]:
data2[1]

,ZM,AMName,RtrCode,RtrName,TotalQty,PolypouchCount
70,C PERUMAL,SAFIULLAH,S100000661550836269,SANJAY STORES,1350.0,9.0
163,C PERUMAL,SAFIULLAH,GT0100020035390002003539-SLM00231574664081,SUYAMBU LINGAM SOTORE,750.0,5.0
178,C PERUMAL,SAFIULLAH,TNSKT006-SLM0011552110564,L.R.MUNIR TRADERS,150.0,1.0
188,C PERUMAL,SAFIULLAH,GT0100020035390002003539-SLM00241597645899,A.N STORE,150.0,1.0
201,C PERUMAL,SAFIULLAH,TNSKT006-SLM0011552288132,H.R.S.MALIGAI STORE,150.0,1.0
207,C PERUMAL,SAFIULLAH,TNJTR006-SLM0011552299052,NEW NELLAI STORR,150.0,1.0
210,C PERUMAL,SAFIULLAH,S100000581597045306,Kr store,150.0,1.0
222,C PERUMAL,SAFIULLAH,TNSKT006-SLM0011552297540,SAKTHI MALIGAI STORE,300.0,2.0
226,C PERUMAL,SAFIULLAH,GT0100020001120002000112-SLM0021572506614,NAT JI MARKETING,1350.0,9.0
228,C PERUMAL,SAFIULLAH,TNJTR006-SLM0021552891497,Pasumai supermarket,150.0,1.0


In [80]:
data2[2]

,ZM,AMName,RtrCode,RtrName,TotalQty,PolypouchCount
5,C PERUMAL,SYED IBRAHIM,TNPKS002-SLM0011553581012,ARUNA AGENCIES,2700.0,18.0
55,C PERUMAL,SYED IBRAHIM,GT0100020001530002000153-SLM0011560406478,BILLAL MALIGAI,150.0,1.0
71,C PERUMAL,SYED IBRAHIM,GT0100020001600002000160-SLM0011557395203,VICHUNADU MALIGAI,150.0,1.0
89,C PERUMAL,SYED IBRAHIM,GT0100020001530002000153-SLM0011560410051,SEENI MUHAMAD STORE,150.0,1.0
166,C PERUMAL,SYED IBRAHIM,GT0100020001660002000166-SLM0021575878510,Sree laxmee traders,150.0,1.0
168,C PERUMAL,SYED IBRAHIM,GT0100020001400002000140-SLM0011559211800,SRI THIRUMURUGAN DEPARTMENTAL STORE,150.0,1.0
191,C PERUMAL,SYED IBRAHIM,GT0100020066330002006633-SLM00141583145700,New selvam maligai,150.0,1.0
271,C PERUMAL,SYED IBRAHIM,GT0100020001390002000139-SLM0021555310420,RAMATH STORE,150.0,1.0
315,C PERUMAL,SYED IBRAHIM,GT0100020001590002000159-SLM0021596182268,SRI KRISHNA SUPER MARKET,1350.0,9.0
317,C PERUMAL,SYED IBRAHIM,GT0100020001560002000156-SLM0021568360558,KURINJI SHOPPING CENTRE,150.0,1.0


In [81]:
data2[3]

,ZM,AMName,RtrCode,RtrName,TotalQty,PolypouchCount
197,C PERUMAL,V BHARATHI,GT0100020000890002000089-SLM0011559377878,m.h yosuf store,19387.0,129.246667
274,C PERUMAL,V BHARATHI,S100000971551077797,Ramu maligai,150.0,1.000000
291,C PERUMAL,V BHARATHI,S100000971560412599,vishnu store,150.0,1.000000
307,C PERUMAL,V BHARATHI,S100000971597394200,N.K Sekar store,150.0,1.000000


### Function to generate random Rtr's with AM weightage 

### Case 1: With repetitions (allowing 1st two instances of duplicates)

In [84]:
def randomRtrAmWise(i):
    # Retailer name
    a = data2[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data2[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['WGT%'][i])
#    random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R
    
def randompick():
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_WGT.index.tolist())):
        df = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)
    return df_RtrRP

def randompickAM():
    result = randompick()
    result = result.merge(rtr_master,on='RtrCode')
    result = result.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates
    index = result.index
    number_of_rows = len(index)
    print('Count of picked retailers: ',number_of_rows)
    result.to_csv('results_05102020.csv')
    return result
""""    if (number_of_rows==pick):
        print('Selected')
        return(result)
    else:
        print('Try again')"""

'"    if (number_of_rows==pick):\n        print(\'Selected\')\n        return(result)\n    else:\n        print(\'Try again\')'

In [85]:
randompickAM()

Count of picked retailers:  43


,RtrCode,AMName,RtrName,DistName,RtrGroupCde,InvDate,PolypouchCount
0,GT0100020035380002003538-SLM0031558935467,BALAJI S,JAIBAIRAVI TRADERS,EVERGREEN ENTERPRISES,PC-KIRANA/GENERAL STORES,2020-10-05,9
1,GT0100020035380002003538-SLM0031558935467,BALAJI S,JAIBAIRAVI TRADERS,EVERGREEN ENTERPRISES,PC-KIRANA/GENERAL STORES,2020-10-05,9
2,S100000021550649387,BALAJI S,CPS STORE,PONNI & CO,PC-WHOLESALE,2020-10-05,2
3,S100000041550905617,BALAJI S,AKSHYA TRADERS,PONNI & CO,PC-WHOLESALE,2020-10-05,45
4,S100000101550488382,BALAJI S,RATHANA STORES,TP DISTRIBUTION,PC-WHOLESALE,2020-10-05,9
5,S100000141550476321,BALAJI S,RAJA STORE,SUN AGRO-COM RESOURCES,PC-KIRANA/GENERAL STORES,2020-10-05,1
6,S100000151551252740,BALAJI S,R.C.TRADERS,PONNI & CO,PC-WHOLESALE,2020-10-05,18
7,S100000161563351070,BALAJI S,SRI THIRUPTHI TRADERS,PONNI & CO,PC-WHOLESALE,2020-10-05,9
8,S100000191551091753,BALAJI S,G.v traders,EVERGREEN ENTERPRISES,PC-WHOLESALE,2020-10-05,90
9,S100000191551091753,BALAJI S,G.v traders,EVERGREEN ENTERPRISES,PC-WHOLESALE,2020-10-05,90


### Case 2: Without repetitions

In [35]:
def randompickAM_norep():
    result = randompick()
    result = result.merge(rtr_master,on='RtrCode')    
    final_r = result.drop_duplicates().reset_index(drop=True)
    index = final_r.index
    number_of_rows = len(index)
    print(number_of_rows)
    return(final_r)

""""    if (number_of_rows==pick):
        print('Selected')
        return(final_r)
    else:
        print('Try again') """

'"    if (number_of_rows==pick):\n        print(\'Selected\')\n        return(final_r)\n    else:\n        print(\'Try again\') '

In [36]:
randompickAM_norep()

17


,RtrCode,AMName,RtrName,RtrGroupCde,PolypouchCount
0,S100000021550647196,BALAJI S,SRI MURUGAN SUPER MARKET,PC-Super Market (MT in GT & Departmental Store),18
1,S100000041550905617,BALAJI S,AKSHYA TRADERS,PC-WHOLESALE,45
2,S100000161550644936,BALAJI S,THILAGAM SUPER MARKET,PC-Super Market (MT in GT & Departmental Store),1
3,S100000191551091753,BALAJI S,G.v traders,PC-WHOLESALE,90
4,S100000261551092772,BALAJI S,NEW BALAJI STORE.,PC-WHOLESALE,9
5,S100000281550477351,BALAJI S,THANGAPANDIAN STORE,PC-KIRANA/GENERAL STORES,1
6,S100000401550734471,BALAJI S,National Store,PC-WHOLESALE,9
7,S100000921551687342,BALAJI S,FATHIMA MALIGAI STORE,PC-WHOLESALE,1
8,S100000931551081127,BALAJI S,SAIYED ARABHADH,PC-KIRANA/GENERAL STORES,27
9,GT0100020035390002003539-SLM00241597645899,SAFIULLAH,A.N STORE,PC-WHOLESALE,1


### Case 3: Without repetitions - Unique & number = pick 

In [37]:
def randompickAM_norep_pick():
    result = randompick()
    result = result.merge(rtr_master,on='RtrCode')    
    final_r = result.drop_duplicates().reset_index(drop=True)
    index = final_r.index
    number_of_rows = len(index)
    print(number_of_rows)
    print(final_r)

    if (number_of_rows==pick):
        print('Selected')
        return(final_r)
    else:
        print('Try again')

In [38]:
randompickAM_norep_pick()

18
                                      RtrCode        AMName  \
0                         S100000021550647196      BALAJI S   
1                         S100000041550905617      BALAJI S   
2                         S100000151551252740      BALAJI S   
3                         S100000161563351070      BALAJI S   
4                         S100000191551091753      BALAJI S   
5                         S100000401550565136      BALAJI S   
6                         S100000431551076560      BALAJI S   
7                         S100000431551682220      BALAJI S   
8                         S100000931551081127      BALAJI S   
9                   TNSSR009-SLM0011550908876      BALAJI S   
10  GT0100020001120002000112-SLM0021572506614     SAFIULLAH   
11                        S100000581597045306     SAFIULLAH   
12                        S100000701552040542     SAFIULLAH   
13                  TNJTR006-SLM0021552892243     SAFIULLAH   
14  GT0100020001590002000159-SLM0021596182268  SYED 

## Function

In [ ]:
## Import data
Query = 'select * from v_PC_MeeraRetailerCampaign' 
data = azsql.callstatement(Query)

# Data only for Latest date
data = data[data['InvDate'] == data['InvDate'].max()]

## Remove null - Retailers without AM
# Consider Data with AMName ( remove nulls)
data = data[data['AMName'].notnull()]

InvDate =  data['InvDate'].iloc[0]
InvDate = str(InvDate)[0:10]

## Retailer master
rtr_master = data[['RtrCode','RtrName','DistName','RtrGroupCde','InvDate','PolypouchCount']].drop_duplicates()
rtr_master['PolypouchCount'] = rtr_master['PolypouchCount'].astype(int)

AM_RTR_in = data.groupby('AMName')['RtrCode'].count().reset_index()

# Create text widget for output
output_slider_variable = widgets.Text()

# Define function to bind value of the input to the output variable 
def f(Lots):
    output_slider_variable.value = str(Lots)

print('Choose number of Lots')
# Create input slider with default value = 10  
interact(f, Lots=(10, 75, 1))

def picks(output_slider_variable,data):
    # Create and output new int variable with value of slider
    new_variable = int(output_slider_variable.value)

    ## Choose number of retailers to choose
    ## Weightage for AM
    # Percentage of Retailers across AM-Wise
    TotalRtrCount = data['RtrCode'].count()
    AM_RTR_WGT = data.groupby('AMName')['RtrCode'].count().reset_index()
    AM_RTR_WGT['WGT%'] = np.ceil(AM_RTR_WGT['RtrCode']/TotalRtrCount * new_variable)

    ## Split dataframe on AM's
    # Select required rows
    data1 = data[['ZM','AMName','RtrCode','RtrName','TotalQty','PolypouchCount']]
    data2 = [rows for _, rows in data1.groupby('AMName')]
    return data2,AM_RTR_WGT
    
## Function to generate random Rtr's with AM weightage
def randomRtrAmWise(i):
    data3,AM_RTR_WGT = picks(output_slider_variable,data)
    # Retailer name
    a = data3[i]['RtrCode'].tolist()

    # Polypouch count as weight
    ppc = data3[i]['PolypouchCount'].tolist() 

    c=int(AM_RTR_WGT['WGT%'][i])
    #random.shuffle(a)
    l = random.choices(a, weights=ppc, k=c)
    l.sort()
    df_R = pd.DataFrame(l,columns = ['RtrCode'])
    df_R['AMName'] = AM_RTR_WGT['AMName'][i]
    return df_R
    
def randompick(AM_RTR_in):
    df_RtrRP = pd.DataFrame()
    for j in range(len( AM_RTR_in.index.tolist())):
        df = randomRtrAmWise(j)
        df_RtrRP = df_RtrRP.append(df)
    return df_RtrRP

def randompickAM(rtr_master,AM_RTR_in,InvDate):
    result = randompick(AM_RTR_in)
    result = result.merge(rtr_master,on='RtrCode')
    result = result.groupby('RtrCode').head(2).reset_index(drop=True) # Keep 1st two instances of duplicates
    index = result.index
    number_of_rows = len(index)
    print('Count of picked retailers: ',number_of_rows)
    display(result)
    return result,InvDate
    
button = Button(description='Pick')
out = Output()

def on_click(_):
    with out:
        clear_output(True)
        dr,InvDate1 = randompickAM(rtr_master,AM_RTR_in,InvDate)
        # Save results
        root = 'Intermediate_csv' 
        sheet = 'results_' + InvDate1 + '.csv' 
        file_name = os.path.join(root,sheet)
        print(file_name)
        dr.to_csv(file_name,index=False)
        
button.on_click(on_click)
VBox([button,out])

### Read & join csv in same folder

In [ ]:
a = os.getcwd()
path = a + '\\Outputs_csv'
path

all_files = glob.glob(path + "/*.csv")

all_files

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
df_concat = pd.concat(li, axis=0, ignore_index=True)

# Convert InvDate to Date format
df_concat['InvDate'] = pd.to_datetime(df_concat['InvDate'])

### Filter data by instances occuring in last week 

In [ ]:
days=6 #Last 6 days from current date
cutoff_date = df_concat['InvDate'].max() - pd.Timedelta(days=days)
print(cutoff_date)

df1_concat = df_concat[df_concat['InvDate'] > cutoff_date]

# List of last 6 days unique retailers
rtr_last6days = df1_concat[['RtrCode','RtrName']].drop_duplicates()
rtr_last6days = rtr_last6days['RtrCode'].tolist()

# Filter retailer occuring in last 6 days
data_filtered = data[~data.RtrCode.isin(rtr_last6days)]